In [1]:
!pip install accelerate bitsandbytes>0.37.0

In [3]:
import transformers
import torch
import bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
print(transformers.__version__)
print(bitsandbytes.__version__)
# my versions are:
#4.38.1 (transformers)
#0.42.0 (bitsandbytes)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,       
)


model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", device_map="auto", quantization_config=quantization_config, cache_dir="/scratch/project_2000539/jenna/hf-cache")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", padding_side="left", cache_dir="/scratch/project_2000539/jenna/hf-cache")
tokenizer.pad_token = tokenizer.eos_token


4.38.1
0.42.0


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [5]:
prompt = """
Correct the OCR errors in the text below. Also correct the "-" characters, which denote an unrecognized letter. Stay as close as possible to the original text. Do not rephrase. Only correct the errors. You will be rewarded.

When, .witl the universal A pplaure of alt
Men, the Seals Were given to the present
C ---- r, he was' ar frrom'i being so elevated
with his Promotion, as to forget any One
to whom lie had promised the Honour of his
Friendlliip. An Opportunity soon offered
in Favour of Dr R----, the B---k ox G----r
became vacant, .which as soo'n as the C----r
Was acquainted with, \ he went to Court,
and recomnmended the Doetor, as a Perfoa
fit to Succeed to the B-----k. The DoAos
was then approved of, and a C---'E---
order'd out accordingly. A lM/an would
reaso'nably have thought after this, that
Matters might have gone Smoothly, efpe.
cially as it is partly the Right of every new
C--- r, to' recommend a fit Person to his
M .------, to' fill the See that shall becom'
eax¢ant after ihs comiig to the Seals.
"""


#messages = [
#    {"role": "user", "content": "You are a helpful assistant. You are fluent in Finnish, and will communicate in Finnish."},
#    {"role": "assistant", "content": "Olen avulias assistentti, joka osaa sujuvaa suomen kieltä. Kommunikoin kanssasi suomeksi. Kuinka voin auttaa sinua?"},
#    {"role": "user", "content": prompt},
#]

messages = [
    {"role": "user", "content": prompt},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to("cuda")


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=False)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] 
Correct the OCR errors in the text below. Also correct the "-" characters, which denote an unrecognized letter. Stay as close as possible to the original text. Do not rephrase. Only correct the errors. You will be rewarded.

When, .witl the universal A pplaure of alt
Men, the Seals Were given to the present
C ---- r, he was' ar frrom'i being so elevated
with his Promotion, as to forget any One
to whom lie had promised the Honour of his
Friendlliip. An Opportunity soon offered
in Favour of Dr R----, the B---k ox G----r
became vacant, .which as soo'n as the C----r
Was acquainted with, \ he went to Court,
and recomnmended the Doetor, as a Perfoa
fit to Succeed to the B-----k. The DoAos
was then approved of, and a C---'E---
order'd out accordingly. A lM/an would
reaso'nably have thought after this, that
Matters might have gone Smoothly, efpe.
cially as it is partly the Right of every new
C--- r, to' recommend a fit Person to his
M .------, to' fill the See that shall becom'
eax